In [1]:
# !pip install cmake

In [10]:
%%cmd
where python

Microsoft Windows [Version 10.0.22621.674]
(c) Microsoft Corporation. All rights reserved.

(base) C:\Users\priya\OneDrive\Desktop\Face recognition>where python
C:\Users\priya\anaconda3\python.exe
C:\Program Files\Python310\python.exe
C:\Users\priya\AppData\Local\Microsoft\WindowsApps\python.exe

(base) C:\Users\priya\OneDrive\Desktop\Face recognition>

In [4]:
%%cmd
python --version

Microsoft Windows [Version 10.0.22621.674]
(c) Microsoft Corporation. All rights reserved.

(base) C:\Users\priya\OneDrive\Desktop\Face recognition>python --version
Python 3.9.12

(base) C:\Users\priya\OneDrive\Desktop\Face recognition>

In [7]:
!pip install "C:\Users\priya\OneDrive\Desktop\Face recognition\Dlib-python whl packages\dlib-19.22.99-cp39-cp39-win_amd64.whl"

Processing c:\users\priya\onedrive\desktop\face recognition\dlib-python whl packages\dlib-19.22.99-cp39-cp39-win_amd64.whl


In [8]:
!pip install face-recognition

  Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)


In [ ]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime

path="photos"
images=[]
classNames=[]
myList=os.listdir(path)
print(myList)
for cl in myList:
    curImg=cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])
print(classNames)

def findEncodings(images):
    encodeList=[]
    for img in images:
        img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
        encode=face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

def markAttendence(name):
    with open('Attendence.txt','r+') as f:
        myDataList=f.readlines()
        nameList=[]
        for line in myDataList:
            entry=line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now =datetime.now()
            dateString=now.strftime('%Y-%m-%d')
            timeString=now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dateString},{timeString}')

encodeListKnown=findEncodings(images)
print('Encoding complete')

cap=cv2.VideoCapture(0)

while True:
    success,img=cap.read()
    imgS=cv2.resize(img,(0,0),None,0.25,0.25)
    imgS=cv2.cvtColor(imgS,cv2.COLOR_BGR2RGB)
    
    facesCurFrame=face_recognition.face_locations(imgS)
    encodesCurFrame=face_recognition.face_encodings(imgS,facesCurFrame)
    
    for encodeFace,faceLoc in zip(encodesCurFrame,facesCurFrame):
        matches=face_recognition.compare_faces(encodeListKnown,encodeFace)
        faceDis= face_recognition.face_distance(encodeListKnown,encodeFace)
#         print(faceDis)
        matchIndex=np.argmin(faceDis)
        
        if matches[matchIndex]:
            name=classNames[matchIndex].upper()
#             print(name)
            y1,x2,y2,x1=faceLoc
            y1,x2,y2,x1=y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(img,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img,(x1,y2-35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img,name,(x1+6,y2-6),cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            markAttendence(name)
            
            
    cv2.imshow('webcam',img)
    if cv2.waitKey(1) & 0xFF==ord('q'): # quit when 'q' is pressed
        cap.release()
        break
cv2.destroyAllWindows() 
cv2.waitKey(1)

['anurag.jpeg', 'bill gates.jpg', 'dr ravindra bhatt.jpg', 'elon musk.jpg', 'garv.jpeg', 'kshitij.jpeg', 'priyansh.jpeg', 'rhythmmm.jpeg']
['anurag', 'bill gates', 'dr ravindra bhatt', 'elon musk', 'garv', 'kshitij', 'priyansh', 'rhythmmm']
Encoding complete
